In [18]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import re
import datetime
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


path = '../../data/socrates/'
files = [ (match[0],match[1]) for f in listdir(path) if isfile(join(path, f))  if (match:=re.search('^socrates_([0-9]{14})\.csv(\.gz)?$', f))]
files

# Build single dataset
odf = pd.DataFrame()
for file,date in files:
    tmp_df = pd.read_csv(path + file)
    odf = pd.concat([odf,tmp_df])

In [22]:
# making a list of active satellites [+] with their max_prob and tca_time

sdf = odf
sdf = sdf[sdf.sat1_name.str.contains("[+]")]
sdf = sdf[sdf.max_prob != 1]
sdf = sdf[sdf.rel_velo_kms > 0.2]
sdf = sdf.drop_duplicates(subset=['sat1_norad','tca_time'])
sdf = sdf.sort_values(by="max_prob", ascending=False)
sdf.columns = sdf.columns.str.replace("sat1","sat")
sdf = sdf.drop(columns=sdf.columns[sdf.columns.str.contains("sat2")])
sdf

single_max_prob = sdf

sdf = odf
sdf = sdf[sdf.sat2_name.str.contains("[+]")]
sdf = sdf[sdf.max_prob != 1]
sdf = sdf[sdf.rel_velo_kms > 0.2]
sdf = sdf.sort_values(by="max_prob", ascending=False)
sdf.columns = sdf.columns.str.replace("sat2","sat")
sdf = sdf.drop(columns=sdf.columns[sdf.columns.str.contains("sat1")])
sdf

single_max_prob = pd.concat([single_max_prob, sdf]).sort_values(by="max_prob", ascending=False)
single_max_prob['tca_time'] = single_max_prob['tca_time'].astype('datetime64[ns]')
single_max_prob = single_max_prob.drop_duplicates(subset=['sat_norad'])
single_max_prob = single_max_prob[["sat_norad","sat_name","max_prob","min_rng_km","rel_velo_kms","tca_time"]]

single_max_prob

,sat_norad,sat_name,max_prob,min_rng_km,rel_velo_kms,tca_time
0,20580,HST [+],9.830000e-02,0.045,8.846,2020-12-15 02:47:18.242
1003,44495,BRO-1 [+],9.830000e-02,0.045,8.846,2020-12-15 02:47:18.242
1007,45409,STARLINK-1282 [+],7.163000e-02,0.060,12.081,2020-12-20 19:15:54.030
0,33320,HUANJING 1A (HJ-1A) [+],5.741000e-02,0.008,1.634,2020-12-19 05:18:29.602
0,37389,X-SAT [+],5.453000e-02,0.020,2.265,2020-12-21 19:28:44.328
...,...,...,...,...,...,...
2010,47168,STARLINK-1885 [+],9.244000e-08,4.770,14.335,2020-12-25 00:02:05.723
2630,46564,STARLINK-1733 [+],8.710000e-08,4.604,14.743,2020-12-29 02:32:07.989
2092,42862,FLOCK 2K-07 [+],8.575000e-08,4.378,15.051,2020-12-12 12:19:25.015
2764,36121,YAOGAN 8 [+],6.767000e-08,4.679,14.453,2020-12-19 03:04:25.907


In [23]:
# get the top 50 to play with
top_50_norad_ids = single_max_prob.sat_norad.head(50).values
as_string = ",".join(map(str,top_50_norad_ids))

In [24]:
f"https://www.space-track.org/basicspacedata/query/class/gp_history/NORAD_CAT_ID/{as_string}/EPOCH/>now-380/orderby/EPOCH desc/limit/100000/format/csv/emptyresult/show"

'https://www.space-track.org/basicspacedata/query/class/gp_history/NORAD_CAT_ID/20580,44495,45409,33320,37389,43123,45701,45762,46354,25114,43923,45233,45551,46771,46549,25544,35498,26702,46336,43689,43657,36834,39763,42962,27424,44885,45732,45061,42958,25985,46150,45362,41992,44100,43078,46556,46284,45225,46052,45381,43195,45783,45234,43256,46051,45754,45092,44752,43144,42010/EPOCH/>now-380/orderby/EPOCH desc/limit/100000/format/csv/emptyresult/show'

In [25]:
df = pd.read_csv("top50_max_prob_socrates.csv")
df['EPOCH'] = df['EPOCH'].astype('datetime64[ns]')
df = df[["NORAD_CAT_ID","INCLINATION","SEMIMAJOR_AXIS","EPOCH"]]

In [52]:
new_df = df.merge(single_max_prob,how="left",left_on="NORAD_CAT_ID",right_on="sat_norad")
new_df["delta"] = new_df.EPOCH - new_df.tca_time
new_df["ddelta"] = new_df.delta / datetime.timedelta(days=1)
new_df = new_df[(new_df.delta > datetime.timedelta(days=-14)) & (new_df.delta < datetime.timedelta(days=7))]
new_df = new_df.sort_values(by="max_prob", ascending=False)

In [53]:
new_df.head(9)

,NORAD_CAT_ID,INCLINATION,SEMIMAJOR_AXIS,EPOCH,sat_norad,sat_name,max_prob,min_rng_km,rel_velo_kms,tca_time,SEMIMAJOR_AXIS_PERCENT_STD,INCLINATION_PERCENT_STD,delta,ddelta
3698,44495,45.0192,6918.443,2020-12-11 03:00:53.809920,44495,BRO-1 [+],0.0983,0.045,8.846,2020-12-15 02:47:18.242,0.000368,0.002601,-4 days +00:13:35.567920,-3.990561
2481,44495,45.0193,6918.404,2020-12-19 04:47:27.385728,44495,BRO-1 [+],0.0983,0.045,8.846,2020-12-15 02:47:18.242,0.000368,0.002601,4 days 02:00:09.143728,4.083439
3958,44495,45.0179,6918.445,2020-12-09 17:55:12.061632,44495,BRO-1 [+],0.0983,0.045,8.846,2020-12-15 02:47:18.242,0.000368,0.002601,-6 days +15:07:53.819632,-5.369516
3937,20580,28.4692,6915.841,2020-12-09 21:34:19.851168,20580,HST [+],0.0983,0.045,8.846,2020-12-15 02:47:18.242,0.000244,0.002518,-6 days +18:47:01.609168,-5.217342
2172,44495,45.0204,6918.392,2020-12-21 03:07:02.985888,44495,BRO-1 [+],0.0983,0.045,8.846,2020-12-15 02:47:18.242,0.000368,0.002601,6 days 00:19:44.743888,6.013712
3898,20580,28.4692,6915.840,2020-12-10 02:36:10.859328,20580,HST [+],0.0983,0.045,8.846,2020-12-15 02:47:18.242,0.000244,0.002518,-6 days +23:48:52.617328,-5.007724
3894,44495,45.0180,6918.444,2020-12-10 03:10:05.187072,44495,BRO-1 [+],0.0983,0.045,8.846,2020-12-15 02:47:18.242,0.000368,0.002601,-5 days +00:22:46.945072,-4.984179
3893,44495,45.0180,6918.444,2020-12-10 03:10:05.187072,44495,BRO-1 [+],0.0983,0.045,8.846,2020-12-15 02:47:18.242,0.000368,0.002601,-5 days +00:22:46.945072,-4.984179
3885,20580,28.4691,6915.839,2020-12-10 04:16:33.221856,20580,HST [+],0.0983,0.045,8.846,2020-12-15 02:47:18.242,0.000244,0.002518,-5 days +01:29:14.979856,-4.938021


Starlink movements
===

These are starlinks that appeared kinda near the top of the `max_prob` filter

Here you can clearly see the starlinks maneuvered, but the 4 that moved were doing so in the same pattern, and wasn't the highest probability to get hit.  some with higher probability didn't move at all (or maybe the other satellite moved?) anyways, not sure here - some maneuvers happened BEFORE socrates' 7 day window - maybe SpaceX has their own prediction that is earlier than 7 days.  We can't conclude they were just maneuvering satellites into where they want.

In [28]:
fig = px.line(new_df[new_df.sat_name.str.contains("STARLINK")], x="ddelta", y="SEMIMAJOR_AXIS", color="sat_name", hover_name="max_prob")
fig.update_yaxes(range=[6924.5, 6928.5])
fig.add_shape(type="line",
    x0=-7, y0=0, x1=-7, y1=20000,
    line=dict(color="gray",width=1)
)

fig.show()

In [29]:
fig = px.line(new_df[new_df.sat_name.str.contains("STARLINK")], x="ddelta", y="INCLINATION", color="sat_name", hover_name="max_prob")
fig.update_yaxes(range=[52.99, 53.06])
fig.add_shape(type="line",
    x0=-7, y0=0, x1=-7, y1=20000,
    line=dict(color="gray",width=1)
)
fig.show()

In [30]:
pstd = new_df.groupby(by="NORAD_CAT_ID")[["SEMIMAJOR_AXIS","INCLINATION"]].agg([('PERCENT_STD', lambda value: 100* value.std() / value.mean())])
pstd.columns = pstd.columns.droplevel(1)
pstd = pstd.rename(columns={'SEMIMAJOR_AXIS':'SEMIMAJOR_AXIS_PERCENT_STD','INCLINATION':'INCLINATION_PERCENT_STD'})
pstd
# cdm_df = cdm_df.merge

single_max_prob = single_max_prob.merge(pstd,how="left",left_on="sat_norad",right_index=True)
single_max_prob = single_max_prob.sort_values("SEMIMAJOR_AXIS_PERCENT_STD", ascending=False)

Single satellite charts
===

I used an iterator here so we can quickly generate charts in jupyter by running the cell following that over and over.
Reset the iterator to start over from the highest `max_prob`

In [31]:
iter_sat_ids = iter(single_max_prob.sat_norad.values)

In [45]:
satellite = next(iter_sat_ids)
plot_df = new_df[new_df.sat_norad == satellite]
sat_name = plot_df.head(1).sat_name
sapstd = single_max_prob[single_max_prob.sat_norad == satellite].SEMIMAJOR_AXIS_PERCENT_STD.values[0]
ipstd = single_max_prob[single_max_prob.sat_norad == satellite].INCLINATION_PERCENT_STD.values[0]

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=plot_df.ddelta, y=plot_df.SEMIMAJOR_AXIS, mode='lines', name="SEMIMAJOR_AXIS"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=plot_df.ddelta, y=plot_df.INCLINATION, mode='lines', name="INCLINATION"),
    secondary_y=True,
)

fig.update_layout(
    title_text=f"Detecting collision avoidance maneuver: {sat_name.values[0]} ({satellite})  Num TLEs: {len(plot_df)}<br>SEMIMAJOR_AXIS %std: {sapstd}%  INCLINATION %std: {ipstd}%"
)

# Set x-axis title
fig.update_xaxes(title_text="Day delta with Time of Closest Approach")

# Set y-axes titles
fig.update_yaxes(title_text="SEMIMAJOR_AXIS", secondary_y=False)
fig.update_yaxes(title_text="INCLINATION", secondary_y=True)

fig.show()

In [ ]:
df = odf

df = df[df.max_prob != 1]
df = df[df.rel_velo_kms > 0.2]
df = df[df.sat1_name.str.contains("[+]") | df.sat2_name.str.contains("[+]")]
df = df.drop_duplicates(subset=['sat1_norad','sat2_norad','extract_date'])
df = df.sort_values(by="max_prob", ascending=False)
df.head(5)


In [ ]:

# sat1 = 43923
# sat2 = 35359
# sat1 = 17369
# sat2 = 42962
# sat1 = 18187
# sat2 = 43078


# df[(df.sat1_norad == sat1) & (df.sat2_norad == sat2)]

In [ ]:
# eventdf = pd.read_csv(f"{sat1}-{sat2}.csv")
# eventdf['EPOCH'] = eventdf['EPOCH'].astype('datetime64[ns]')
# eventdf = eventdf.set_index("EPOCH")
# eventdf = eventdf[["NORAD_CAT_ID","MEAN_MOTION","ECCENTRICITY","INCLINATION","ARG_OF_PERICENTER","MEAN_ANOMALY",
#     "BSTAR","MEAN_MOTION_DOT","SEMIMAJOR_AXIS","PERIOD","APOAPSIS","PERIAPSIS"]]

In [ ]:
# eventdf = eventdf[eventdf.index.to_series().between('2020-1-1', '2021-1-1')]
# eventdf[eventdf.NORAD_CAT_ID==sat1].plot(subplots=True,figsize=(5,20));
# eventdf[eventdf.NORAD_CAT_ID==sat2].plot(subplots=True,figsize=(5,20));

In [ ]:
# eventdf = eventdf[eventdf.index.to_series().between('2020-12-09', '2020-12-16')]

# eventdf[eventdf.NORAD_CAT_ID==sat1].plot(subplots=True,figsize=(5,20));
# eventdf[eventdf.NORAD_CAT_ID==sat2].plot(subplots=True,figsize=(5,20));